In [1]:
import torch
from transformers import GPT2Tokenizer
from myllm_model import MyModel
from torch import nn

In [2]:
tkn = GPT2Tokenizer.from_pretrained('./tokenizer/')
tkn.pad_token = '[PAD]'
tkn.add_tokens(['[END]'])
VOCAB_SIZE = tkn.vocab_size

max_len = 512

model = MyModel(
    vocab=VOCAB_SIZE,
    pad_token_id=tkn.pad_token_id,
    d_model=2560,
    num_head=32,
    num_block=24,
    max_len=max_len
)

In [3]:
chkpt = torch.load('/root/myllm3-2B-103200.pt')
model.load_state_dict(chkpt['module'])

<All keys matched successfully>

In [4]:
max_len_2 = 1024
pos2 = nn.Embedding(max_len_2, model.pos.weight.shape[1])
with torch.no_grad():
    pos2.weight[:max_len] = model.pos.weight.clone()
    
assert torch.sum(pos2.weight[:max_len] - model.pos.weight).item() == 0

In [5]:
model.pos = pos2
for b in model.blocks:
    b.attn.register_buffer(
            'mask', 1 - torch.triu(torch.ones((1, 1, max_len_2, max_len_2)), diagonal=1))

In [6]:
torch.save({
    'module': model.half().state_dict()
}, '/root/autodl-tmp/myllm4-2B-103200.pt')